<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
import pandas as pd
import sys
import time
sys.path.append("..")

from utils import my_print

In [83]:
N_REPS = 1
DATA_DIR = "../data/"
FILE_NAME = "complaint_census_join.csv"
OUTPUT_NAME = "complaint_census_join_by_zip_day_pandas_version.out"
TESTING = False

if TESTING:
    FILE_NAME = "complaint_census_join_sample.csv"

base_cols = ['zcta', 'date', 'type', 'descriptor', 'geoID']
num_cols = ['median_earning', 'full_time_median_earning', 
          'full_time_mean_earning']
header = base_cols + num_cols

all_times = []
for i in range(N_REPS):
    start_time = time.time()
    past_time = my_print(iteration=i, text="Start time epoch", 
                         past_time=start_time, use_duration=False)
    # read data
    df = pd.read_csv(
        DATA_DIR+FILE_NAME, low_memory=False, names=header,
        dtype={"descriptor": str}
    )
    # This is faster than reading it as a date
    df["date"] = pd.to_datetime(df["date"], format="%m/%d/%Y %H:%M:%S %p")
    past_time = my_print(iteration=i, text="Time taken reading data", past_time=past_time)
    
    # filter time
    final_df = df.query("""
        (date >= '03/01/2019' and date < '11/01/2019') \
        or \
        (date >= '03/01/2020' and date < '11/01/2020') \
                        
    """)
    # we have too little data for the filter to return anything 
    if TESTING:
        final_df = df

    # add binary noise column
    bin_noise = final_df["descriptor"].str.contains("noise", case=False)
    final_df["binary_noise"] = bin_noise.fillna(False).astype(int)

    grouper = final_df.groupby(["zcta", pd.Grouper(freq="1D", key="date")], sort=True)
    # replace the original final_df
    final_df = grouper.first()[[
        "geoID", "median_earning", "full_time_median_earning",
        "full_time_mean_earning", 
    ]]
    final_df["num_complaints"]  = grouper.size()  
    final_df["num_noise_complaints"] = grouper["binary_noise"].sum()
    past_time = my_print(iteration=i, text="Time taken processing data", past_time=past_time)
    
    # write 
    final_df.to_csv(DATA_DIR+OUTPUT_NAME)
    past_time = my_print(iteration=i, text="Time taken saving data", past_time=past_time)
    
    past_time = my_print(iteration=i, text="Total time", past_time=start_time)
    all_times.append(int(past_time-start_time))
    print("\n\n")

Iteration 0: Start time epoch (s): 1607636588.623392
Iteration 0: Time taken reading data (s): 178


/home/bruno/.local/lib/python3.6/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Iteration 0: Time taken processing data (s): 11
Iteration 0: Time taken saving data (s): 0
Iteration 0: Total time (s): 190





In [85]:
178/190

0.9368421052631579

In [86]:
4*60 + 18

258

In [87]:
6*60 + 13

373

In [ ]:
import pyspark
sc = pyspark.SparkContext(appName="some_app")
sc.uiWebUrl